In [1]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

df_e_price = pd.read_csv("../src/cleaned_e_price.csv")
df_emission = pd.read_csv("../src/cleaned_emission_annual.csv")
df_fuel_ratio = pd.read_csv("../src/cleaned_fuel_ratio.csv")

In [2]:
merged = pd.merge(df_e_price, df_emission, on=["Year", "State"], how="inner")
merged = pd.merge(merged, df_fuel_ratio, on=["Year", "State"], how="inner")

In [3]:
merged.head()

,State,Year,e_price,CO2_annual,Fuels_ratio
0,CT,2001,9.62,45649924,0.0
1,ME,2001,10.55,35440016,0.0
2,MA,2001,11.55,99152464,0.0
3,NH,2001,10.95,18182220,0.0
4,RI,2001,11.45,13784508,0.0


In [4]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1071 entries, 0 to 1070
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   State        1071 non-null   object 
 1   Year         1071 non-null   int64  
 2   e_price      1071 non-null   float64
 3   CO2_annual   1071 non-null   int64  
 4   Fuels_ratio  1071 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 50.2+ KB


In [5]:
merged.describe()

,Year,e_price,CO2_annual,Fuels_ratio
count,1071.00000,1071.000000,1.071000e+03,1071.000000
mean,2011.00000,9.811223,1.714041e+08,0.374959
std,6.05813,3.781943,1.713947e+08,0.407917
min,2001.00000,4.240000,2.633200e+04,0.000000
25%,2006.00000,7.465000,4.884882e+07,0.000000
50%,2011.00000,9.020000,1.318715e+08,0.109052
75%,2016.00000,10.770000,2.303515e+08,0.858901
max,2021.00000,34.040000,1.069856e+09,0.998595


In [6]:
merged["Year"].unique(), merged["State"].unique()

(array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
        2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]),
 array(['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA', 'IL', 'IN',
        'MI', 'OH', 'WI', 'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD', 'DE',
        'DC', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'WV', 'AL', 'KY', 'MS',
        'TN', 'AR', 'LA', 'OK', 'TX', 'AZ', 'CO', 'ID', 'MT', 'NV', 'NM',
        'UT', 'WY', 'CA', 'OR', 'WA', 'AK', 'HI'], dtype=object))

In [7]:
# merged.to_csv("../src/merged_data.csv", index=False)

In [8]:
# log transformation of CO2 emission
import numpy as np

merged["CO2_log"] = np.log(merged["CO2_annual"])

In [9]:
joined_rggi = ["CT", "DE", "ME", "MD", "MA", "NH", "NY", "RI", "VT"]

merged["treated"] = merged["State"].isin(joined_rggi)

In [10]:
# select years before and after the treatment
Pre = list(range(2001, 2009))
Post = list(range(2010, 2018))
merged["post"] = merged["Year"].isin(Post)
merged["treated:post"] = merged["treated"] * merged["post"]
merged_subset = merged[merged["Year"].isin(Post + Pre)]

# drop NJ as it initially withdrew in 2012 but rejoined in 2020.
merged_subset = merged_subset[merged_subset["State"] != "NJ"].reset_index(drop=True)

In [11]:
merged_subset

,State,Year,e_price,CO2_annual,Fuels_ratio,CO2_log,treated,post,treated:post
0,CT,2001,9.62,45649924,0.000000,17.636513,True,False,False
1,ME,2001,10.55,35440016,0.000000,17.383352,True,False,False
2,MA,2001,11.55,99152464,0.000000,18.412169,True,False,False
3,NH,2001,10.95,18182220,0.000000,16.715955,True,False,False
4,RI,2001,11.45,13784508,0.000000,16.439056,True,False,False
...,...,...,...,...,...,...,...,...,...
795,CA,2017,16.06,177732004,0.483931,18.995787,False,True,False
796,OR,2017,8.81,31963616,0.001395,17.280109,False,True,False
797,WA,2017,7.94,44030952,0.000000,17.600403,False,True,False
798,AK,2017,19.10,14126064,0.983763,16.463532,False,True,False


In [12]:
merged_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   State         800 non-null    object 
 1   Year          800 non-null    int64  
 2   e_price       800 non-null    float64
 3   CO2_annual    800 non-null    int64  
 4   Fuels_ratio   800 non-null    float64
 5   CO2_log       800 non-null    float64
 6   treated       800 non-null    bool   
 7   post          800 non-null    bool   
 8   treated:post  800 non-null    bool   
dtypes: bool(3), float64(3), int64(2), object(1)
memory usage: 40.0+ KB


In [13]:
import statsmodels.formula.api as smf

model = smf.ols(
    "CO2_log ~ Fuels_ratio + e_price + treated + post + treated:post",
    data=merged_subset,
).fit(
    cov_type="cluster",
    cov_kwds={"groups": merged_subset["State"]},
)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                CO2_log   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     13.72
Date:                Thu, 25 Apr 2024   Prob (F-statistic):           2.21e-08
Time:                        13:36:42   Log-Likelihood:                -1462.3
No. Observations:                 800   AIC:                             2937.
Df Residuals:                     794   BIC:                             2965.
Df Model:                           5                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       18.6270      0.549     33.921      0.000      17.551      19.703
treated[T.True]                 -1.0760      0.853     -1.262      0.207      -2.748       0.596
post[T.True]                     0.0156      0.106      0.146      0.884      -0.193       0.224
treated[T.True]:post[T.True]    -0.2397      0.099     -2.433      0.015      -0.433      -0.047
Fuels_ratio                      1.0901      0.550      1.981      0.048       0.012       2.169
e_price                         -0.0576      0.047     -1.220      0.223      -0.150       0.035
==============================================================================
Omnibus:                      347.400   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1501.537
Skew:                          -2.028   Prob(JB):                         0.00
Kurtosis:                       8.347   Cond. No.                         62.0
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [14]:
merged_subset = merged_subset.set_index(["State", "Year"])

In [15]:
print(merged_subset.index)

MultiIndex([('CT', 2001),
            ('ME', 2001),
            ('MA', 2001),
            ('NH', 2001),
            ('RI', 2001),
            ('VT', 2001),
            ('NY', 2001),
            ('PA', 2001),
            ('IL', 2001),
            ('IN', 2001),
            ...
            ('MT', 2017),
            ('NV', 2017),
            ('NM', 2017),
            ('UT', 2017),
            ('WY', 2017),
            ('CA', 2017),
            ('OR', 2017),
            ('WA', 2017),
            ('AK', 2017),
            ('HI', 2017)],
           names=['State', 'Year'], length=800)


In [16]:
from linearmodels.panel import PanelOLS

formula = "CO2_log ~ Fuels_ratio + e_price + treated + post + treated:post + EntityEffects + TimeEffects"
mod = PanelOLS.from_formula(
    formula,
    data=merged_subset,
    drop_absorbed=True,
).fit(cov_type="clustered", cluster_entity=True)

mod.summary

/var/folders/xs/t_l_jm7548d0gxdvnxpbdfsw0000gp/T/ipykernel_56940/1047470114.py:8: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

treated, post

  ).fit(cov_type="clustered", cluster_entity=True)


Dep. Variable:,CO2_log,R-squared:,0.1047
Estimator:,PanelOLS,R-squared (Between):,0.0211
No. Observations:,800,R-squared (Within):,0.1875
Date:,"Thu, Apr 25 2024",R-squared (Overall):,0.0211
Time:,13:36:43,Log-likelihood,344.25
Cov. Estimator:,Clustered,,
,,F-statistic:,28.546
Entities:,50,P-value,0.0000
Avg Obs:,16.000,Distribution:,"F(3,732)"
Min Obs:,16.000,,
Max Obs:,16.000,F-statistic (robust):,4.9703
